# Overlaid Arrow Image Generator Notebook

To support training and inference of the arrow detector, we want to create a pipeline to generate images with a variety of different arrow PNGs randomnly overlaid onto medical images of varying modalities.

In [2]:
import cv2
import numpy as np
from PIL import Image
import random
import os

In [83]:
def preprocess_arrow(arrow_image, rotation_angle=0, scale_x=None, scale_y=None, color=None):
    """
    Preprocess arrow PNG by rotating, scaling, and coloring it.
    
    Parameters:
    -----------
    arrow_image : np.ndarray
        Arrow image loaded with cv2.IMREAD_UNCHANGED (includes alpha channel if present)
    rotation_angle : float
        Rotation angle in degrees (0 = pointing right, 90 = pointing up, -90 = pointing down, 180 = pointing left)
    scale_x : float, optional
        Horizontal scale factor (default: random between 0.2 and 1.0)
    scale_y : float, optional
        Vertical scale factor (default: random between 0.2 and 1.0)
    color : tuple, optional
        RGB color tuple (0-255) to tint the arrow (default: random color)
    
    Returns:
    --------
    arrow_rgb : np.ndarray
        Rotated and scaled arrow image in RGB format
    alpha : np.ndarray
        Alpha channel for the rotated arrow
    tip_local : tuple
        (x, y) coordinates of the arrow tip in the rotated image
    tail_local : tuple
        (x, y) coordinates of the arrow tail in the rotated image
    """
    # Generate random scale factors if not provided
    if scale_x is None:
        scale_x = random.uniform(0.5, 2.0)
    if scale_y is None:
        scale_y = random.uniform(0.05, 0.5)
    
    # Generate random color if not provided
    if color is None:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    # Extract BGR and alpha from arrow image
    if arrow_image.shape[2] == 4:
        arrow_bgr = arrow_image[:, :, :3]
        alpha = arrow_image[:, :, 3] / 255.0
    else:
        arrow_bgr = arrow_image
        alpha = np.ones((arrow_image.shape[0], arrow_image.shape[1]))
    
    # Convert arrow from BGR to RGB
    arrow_rgb = cv2.cvtColor(arrow_bgr, cv2.COLOR_BGR2RGB)
    
    # Apply random color tint
    # Convert color from RGB to float [0, 1] for multiplication
    color_normalized = np.array(color, dtype=np.float32) / 255.0
    arrow_rgb = (arrow_rgb.astype(np.float32) * color_normalized).astype(np.uint8)
    
    # Apply random scaling (independently for x and y)
    h, w = arrow_rgb.shape[:2]
    new_w = int(w * scale_x)
    new_h = int(h * scale_y)
    
    if new_w > 0 and new_h > 0:
        arrow_rgb = cv2.resize(arrow_rgb, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        alpha = cv2.resize(alpha, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    # Original arrow dimensions and tip/tail positions (before rotation)
    # Tip is in the middle of the right side, tail on the left
    offset = 160 * scale_x
    original_tip_local = (arrow_rgb.shape[1] - offset, arrow_rgb.shape[0] // 2)
    original_tail_local = (offset, arrow_rgb.shape[0] // 2)
    
    # If no rotation needed, return as-is
    if rotation_angle == 0:
        return arrow_rgb, alpha, original_tip_local, original_tail_local
    
    # Get the center of rotation (the tip point)
    h, w = arrow_rgb.shape[:2]
    center = original_tip_local
    
    # Create rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    
    # Calculate new canvas size to fit the rotated image
    # Get the four corners of the original image
    corners = np.array([
        [0, 0, 1],
        [w, 0, 1],
        [0, h, 1],
        [w, h, 1]
    ]).T
    
    # Transform corners to find new bounding box
    transformed = rotation_matrix @ corners
    x_coords = transformed[0, :]
    y_coords = transformed[1, :]
    
    x_min = int(np.floor(x_coords.min()))
    x_max = int(np.ceil(x_coords.max()))
    y_min = int(np.floor(y_coords.min()))
    y_max = int(np.ceil(y_coords.max()))
    
    new_w = x_max - x_min
    new_h = y_max - y_min
    
    # Adjust rotation matrix to account for the new canvas position
    rotation_matrix[0, 2] -= x_min
    rotation_matrix[1, 2] -= y_min
    
    # Rotate the arrow image with the expanded canvas
    arrow_rgb_rotated = cv2.warpAffine(arrow_rgb, rotation_matrix, (new_w, new_h), 
                                        borderMode=cv2.BORDER_CONSTANT, 
                                        borderValue=(0, 0, 0))
    
    # Rotate the alpha channel
    alpha_rotated = cv2.warpAffine(alpha, rotation_matrix, (new_w, new_h), 
                                     borderMode=cv2.BORDER_CONSTANT, 
                                     borderValue=0)
    
    # Transform the tail coordinates using the adjusted rotation matrix
    tail_point = np.array([original_tail_local[0], original_tail_local[1], 1]).reshape(3, 1)
    tail_rotated = rotation_matrix @ tail_point
    tail_local_rotated = (int(round(tail_rotated[0, 0])), int(round(tail_rotated[1, 0])))
    
    # Transform the tip coordinates
    tip_point = np.array([original_tip_local[0], original_tip_local[1], 1]).reshape(3, 1)
    tip_rotated = rotation_matrix @ tip_point
    tip_local_rotated = (int(round(tip_rotated[0, 0])), int(round(tip_rotated[1, 0])))
    
    return arrow_rgb_rotated, alpha_rotated, tip_local_rotated, tail_local_rotated


In [1]:
# Load base medical image
base_image_path = "medical-images/image1.jpg"
base_image = cv2.imread(base_image_path)
base_image_rgb = cv2.cvtColor(base_image, cv2.COLOR_BGR2RGB)

# Load arrow PNG
arrow_path = "arrow-pngs/arrow1.png"
arrow_image = cv2.imread(arrow_path, cv2.IMREAD_UNCHANGED)

# Preprocess arrow with random rotation, scaling, and color
rotation_angle = random.uniform(0, 360)
arrow_rgb, alpha, tip_local, tail_local = preprocess_arrow(arrow_image, rotation_angle)

# Get dimensions
base_h, base_w = base_image_rgb.shape[:2]
arrow_h, arrow_w = arrow_rgb.shape[:2]

# Generate random position for the arrow TIP (prioritize tip staying on screen, tail can go off)
margin = 10  # Small margin from edges
tip_x = random.randint(margin, base_w - margin)
tip_y = random.randint(margin, base_h - margin)

tail_x = tip_x + (tail_local[0] - tip_local[0])
tail_y = tip_y + (tail_local[1] - tip_local[1])

# Calculate where to place the arrow so its tip is at (tip_x, tip_y)
arrow_x = tip_x - tip_local[0]
arrow_y = tip_y - tip_local[1]

arrow_tail_x = tip_x - tail_local[0]
arrow_tail_y = tip_y - tail_local[1]

# Create output image (copy of base)
output = base_image_rgb.copy()

# Blend arrow onto base image using alpha channel
# Calculate the region where arrow overlaps with base
y_start = max(0, arrow_y)
y_end = min(base_h, arrow_y + arrow_h)
x_start = max(0, arrow_x)
x_end = min(base_w, arrow_x + arrow_w)

# Corresponding region in arrow image
arrow_y_start = y_start - arrow_y
arrow_y_end = y_end - arrow_y
arrow_x_start = x_start - arrow_x
arrow_x_end = x_end - arrow_x

# Blend
alpha_region = alpha[arrow_y_start:arrow_y_end, arrow_x_start:arrow_x_end]
for c in range(3):
    output[y_start:y_end, x_start:x_end, c] = (
        output[y_start:y_end, x_start:x_end, c] * (1 - alpha_region) + 
        arrow_rgb[arrow_y_start:arrow_y_end, arrow_x_start:arrow_x_end, c] * alpha_region
    )

# Display result
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
plt.imshow(output)
# Mark the tip location
plt.plot(tip_x, tip_y, 'bo', markersize=15, markeredgewidth=2, label='Tip')
plt.plot(tail_x, tail_y, 'yo', markersize=15, markeredgewidth=2, label='Tail')
plt.title(f"Arrow rotated {rotation_angle:.1f}° - Tip at ({tip_x}, {tip_y})")
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.show()


KeyboardInterrupt

